In [1]:
import sys
import math
import numpy as np
import pickle
from scipy.stats import binom
from scipy.stats import poisson
from scipy.special import binom as choose
import matplotlib.pyplot as plt

In [2]:
Data_Root = '/data/haopeng/Diffusion/SBM/'

In [3]:
def F(m, k, R):
    """ the F function that determines whether a node is over the threshold (R)
    or not. Assuming the same threshold for everyone. """
    if m > R * k:
        return 1
    return 0

def g_sum1(k, qbar, R):
    s = 0.0
    for m in range(k):
        s += F(m, k, R) * choose(k-1, m) * qbar**m * (1.0-qbar)**(k-1-m)
    return s

def h_sum1(k, qbar, R):
    s = 0.0
    for m in range(k+1):
        s += F(m, k, R) * choose(k, m) * qbar**m * (1.0-qbar)**(k-m)
    return s

def g_sum2(Pk, z, r0, qbar, R):
    """ 
    Obtain q_{n+1} from q_n. 

    Pk: a list of tuples that contains (k, pk). 
    z: mean degree
    r0: a list of the seed fraction for each community.

    """
    qnext = []
    for i in range(len(qbar)):
        s = 0.0
        for k, pk in Pk:
            s += k*pk*g_sum1(k, qbar[i], R) / z
        qnext.append(r0[i]+(1.0-r0[i])*s)
    return np.array(qnext)

def h_sum2(Pk, z, r0, qbar, R):
    """ 
    Obtain r_{n+1} from q_n. 

    Pk: a list of tuples that contains (k, pk). 
    z: mean degree
    r0: the seed fraction

    """
    rnext = []
    for i in range(len(qbar)):
        s = 0.0
        for k, pk in Pk:
            s += pk * h_sum1(k, qbar[i], R)
        rnext.append(r0[i] + (1 - r0[i]) * s)
    return np.array(rnext)

In [4]:
def total_fraction(rho):
    return 0.5*rho[0] + 0.5*rho[1]

def cal_qbar(q, e):
    """ only applicable for # comm = 2 """
    qbar = []
    for i in range(len(q)):
        qbar.append((e[i][0] * q[0] + e[i][1] * q[1]) / (e[i][0] + e[i][1]))
    return np.array(qbar)

def non_negative(delta):
    hold = np.zeros_like(delta)
    return np.maximum(delta, hold)

In [5]:
def cal_step_size(num_nodes, Pk, rho0, mu, thsh, frac=0.01):
    """ from Pk and r0, calculate the final density rho using qbar_{\inf}.
        only one community (community 0) is seeded. 
    """
    step_size = [] # frac of active nodes in (1) whole network; (2) C1; (3) C2.
    z = sum(k*pk for k, pk in Pk)
    e = [[(1.0-mu),mu],[mu,(1.0-mu)]]
    q = rho0.copy()
    rho = rho0.copy()
    rho_total_old = total_fraction(rho)
    
    step_size.append(np.array([total_fraction(rho0), rho0[0], rho0[1]]))
    while True:
        qbar = cal_qbar(q, e)
        q_next = g_sum2(Pk, z, rho0, qbar, thsh)
        q = non_negative(q_next - q) * frac + q
        rho_next = h_sum2(Pk, z, rho0, qbar, thsh)
        rho = non_negative(rho_next - rho) * frac + rho
        rho_total = total_fraction(rho)
        if abs(rho_total - rho_total_old)*num_nodes < 1:
            break
        step_size.append(np.array([rho_total, rho[0], rho[1]]))
        rho_total_old = rho_total
    return step_size

def speed_each_step(step_size):
    step_speed = []
    total_step = len(step_size)-1 # e.g., index are 0, 1, 2.
    for i in range(total_step):
        step_speed.append(step_size[i+1] - step_size[i])
    step_speed.append(np.array([0, 0, 0])) # zeros for the last step.
    return step_speed

In [6]:
num_nodes = 100000
avg_degree = 10
num_edges = num_nodes*avg_degree/2
num_community = 2
f = 0.01 #1.0

In [7]:
b = poisson(avg_degree)
Pk = [(k, prob) for (k, prob) in [(k, b.pmf(k)) for k in range(num_nodes)] if prob > 1/num_nodes]

In [8]:
thsh = 0.35
rho0 = np.array([0.2, 0])
us = np.arange(0, 0.51, 0.01)

In [616]:
step_size = cal_step_size(num_nodes, Pk, rho0, 0.5, thsh, frac=f)

In [617]:
step_size[-1]

array([0.99994886, 0.9999503 , 0.99994741])

In [626]:
results_analy = {}

for u in us:
    step_size = cal_step_size(num_nodes, Pk, rho0, u, thsh, frac=f)
    step_speed = speed_each_step(step_size)
    results_analy[u] = (step_size, step_speed)

In [627]:
with open(Data_Root+'analy_nodes_%sk_z_%s_thsh_%s_rho0_%s_f_%s.pickle'%(num_nodes//1000, avg_degree, thsh, rho0[0]/2, f), 'wb') as file:
    pickle.dump(dict(results_analy), file)
    

In [111]:
with open(Data_Root+'analy_nodes_%sk_z_%s_thsh_%s_rho0_%s_f_%s.pickle'%(num_nodes//1000, avg_degree, thsh, rho0[0]/2, f), 'rb') as file:
    results_analy = pickle.load(file)